### Pre-Selection

In [10]:
import pandas as pd
import src.functions as src

features = "dataset1_features.csv"
features = pd.read_csv(features, sep=',', header=0, index_col=None)
Y = features[['labels']]

X_all = features.drop('labels', axis=1)
all_pre= ['3_','4_','6_','7_','10_','11_','12_','15_','16_','18_','25_','26_','27_','28_','29_','30_','33_','34_']
X_all_pre=X_all[[col for col in X_all.columns if any(col.startswith(prefix) for prefix in all_pre)]]
all_post=  ['1_','2_','5_','7_','8_','9_','12_','13_','14_','17_','19_','20_','21_','22_','23_','24_','31_','32_','35_','36_']
X_all_post=X_all[[col for col in X_all.columns if any(col.startswith(prefix) for prefix in all_post)]]
print('All Features:')
print('Total: ' + str(len(X_all.columns))) 
print('Pre: ' + str(len(X_all_pre.columns)))
print('Post: ' + str(len(X_all_post.columns)))

theta_prefixes= ['1_','2_','3_','4_','5_','6_','7_','8_','9_','10_','11_','12_','13_','14_','15_','16_','17_','18_']
X_theta= X_all[[col for col in X_all.columns if any(col.startswith(prefix) for prefix in theta_prefixes)]]
theta_pre= ['3_','4_','6_','7_','10_','11_','12_','15_','16_','18_']
X_theta_pre= X_theta[[col for col in X_theta.columns if any(col.startswith(prefix) for prefix in theta_pre)]]
theta_post= ['1_','2_','5_','7_','8_','9_','12_','13_','14_','17_']
X_theta_post= X_theta[[col for col in X_theta.columns if any(col.startswith(prefix) for prefix in theta_post)]]
print('Theta Features:')
print('Total: ' + str(len(X_theta.columns)))
print('Pre: ' + str(len(X_theta_pre.columns)))
print('Post: ' + str(len(X_theta_post.columns)))

frequency_prefixes= ['1_','2_','3_','4_','5_','6_','7_','8_','9_','10_','11_','12_','13_','14_','15_','16_','17_','18_','19_','20_','21_','22_','23_','24_','25_','26_','27_','28_','29_','30_']
X_frequency= X_all[[col for col in X_all.columns if any(col.startswith(prefix) for prefix in frequency_prefixes)]]
frequency_pre=['3_','4_','6_','7_','10_','11_','12_','15_','16_','18_','25_','26_','27_','28_','29_','30_']
X_frequency_pre= X_frequency[[col for col in X_frequency.columns if any(col.startswith(prefix) for prefix in frequency_pre)]]
frequency_post=['1_','2_','5_','7_','8_','9_','12_','13_','14_','17_','19_','20_','21_','22_','23_','24_']
X_frequency_post= X_frequency[[col for col in X_frequency.columns if any(col.startswith(prefix) for prefix in frequency_post)]]
print('Frequency Features:')
print('Total: ' + str(len(X_frequency.columns)))
print('Pre: ' + str(len(X_frequency_pre.columns)))
print('Post: ' + str(len(X_frequency_post.columns)))

temporal_prefixes= ['31_','32_','33_','34_','35_','36_']
X_temporal= X_all[[col for col in X_all.columns if any(col.startswith(prefix) for prefix in temporal_prefixes)]]
temporal_pre= ['33_','34_']
X_temporal_pre= X_temporal[[col for col in X_temporal.columns if any(col.startswith(prefix) for prefix in temporal_pre)]] 
temporal_post= ['31_','32_','35_','36_']
X_temporal_post= X_temporal[[col for col in X_temporal.columns if any(col.startswith(prefix) for prefix in temporal_post)]]
print('Temporal Features:')
print('Total: ' + str(len(X_temporal.columns)))
print('Pre: ' + str(len(X_temporal_pre.columns)))
print('Post: ' + str(len(X_temporal_post.columns)))




All Features:
Total: 1629
Pre: 844
Post: 846
Theta Features:
Total: 667
Pre: 364
Post: 364
Frequency Features:
Total: 1387
Pre: 724
Post: 724
Temporal Features:
Total: 242
Pre: 120
Post: 122


### Feature Selection

#### Theta Features

In [ ]:
from mrmr import mrmr_classif

selected_features = mrmr_classif(X=X_theta, y=Y, K=30)
combined_df=pd.DataFrame()
for i in selected_features:
    combined_df=pd.concat([combined_df, X_theta[i]], axis=1)
Theta_features=combined_df

#### Frequency Features

In [ ]:
from mrmr import mrmr_classif

selected_features = mrmr_classif(X=X_frequency, y=Y, K=30)
combined_df=pd.DataFrame()
for i in selected_features:
    combined_df=pd.concat([combined_df, X_frequency[i]], axis=1)
Frequency_features=combined_df

#### Temporal Features

In [ ]:
from mrmr import mrmr_classif

selected_features = mrmr_classif(X=X_temporal, y=Y, K=30)
combined_df=pd.DataFrame()
for i in selected_features:
    combined_df=pd.concat([combined_df, X_temporal[i]], axis=1)
Temporal_features=combined_df

#### All Features

In [14]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
import scipy.signal as signal
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score, balanced_accuracy_score, confusion_matrix
import pandas as pd

from mrmr import mrmr_classif

selected_features = mrmr_classif(X=X_all, y=Y, K=30)
combined_df=pd.DataFrame()
for i in selected_features:
    combined_df=pd.concat([combined_df, X_all[i]], axis=1)
All_features=combined_df

X=All_features
i=30
test=0.5
while test>=0.1:
    f1score = []
    precision = []
    recall = []
    specificity = []
    npv = []
    bal_acc = []

    splits = 100
    sss = StratifiedShuffleSplit(n_splits=splits, test_size=test, random_state=0)
    svm = SVC(class_weight='balanced')
    scaler = StandardScaler()
    X_col = X.columns
    Y=np.array(Y)

    for train_index, test_index in sss.split(X, Y):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        Y_train, Y_test = Y[train_index], Y[test_index]
        scal = scaler.fit(X_train)
        X_train = scal.transform(X_train)  # Variables standardization
        X_test = scal.transform(X_test)  # Variables standardization
        X_train = pd.DataFrame(X_train, columns=X_col)
        X_test = pd.DataFrame(X_test, columns=X_col)
        clf = svm.fit(X_train, Y_train)
        Y_predicted = clf.predict(X_test)
        f1score.append(f1_score(Y_test, Y_predicted))
        precision.append(precision_score(Y_test, Y_predicted))  # Precision = Positive predictive value
        npv.append(precision_score(Y_test, Y_predicted, pos_label=0))  # Negative predictive value
        recall.append(recall_score(Y_test, Y_predicted))  # Recall = Sensitivity
        specificity.append(recall_score(Y_test, Y_predicted, pos_label=0))
        bal_acc.append(balanced_accuracy_score(Y_test, Y_predicted))

    print("Number of features used: "+ str(i))
    print("Test Size: {:.2f}".format(test))
    print("Mean Sensitivity: {:.4f}".format(np.mean(recall)))
    print("Std Sensitivity: {:.4f}".format(np.std(recall)))
    print("Mean Specificity: {:.4f}".format(np.mean(specificity)))
    print("Std Specificity: {:.4f}".format(np.std(specificity)))
    print("Mean Balanced Accuracy: {:.4f}".format(np.mean(bal_acc)))
    print("Std Balanced Accuracy: {:.4f}".format(np.std(bal_acc)))
    test=test-0.05

100%|██████████| 30/30 [00:11<00:00,  2.65it/s]


Number of features used: 30
Test Size: 0.50
Mean Sensitivity: 0.3269
Std Sensitivity: 0.0484
Mean Specificity: 0.9206
Std Specificity: 0.0125
Mean Balanced Accuracy: 0.6237
Std Balanced Accuracy: 0.0221
Number of features used: 30
Test Size: 0.45
Mean Sensitivity: 0.3243
Std Sensitivity: 0.0435
Mean Specificity: 0.9215
Std Specificity: 0.0136
Mean Balanced Accuracy: 0.6229
Std Balanced Accuracy: 0.0199
Number of features used: 30
Test Size: 0.40
Mean Sensitivity: 0.3413
Std Sensitivity: 0.0546
Mean Specificity: 0.9168
Std Specificity: 0.0128
Mean Balanced Accuracy: 0.6290
Std Balanced Accuracy: 0.0254
Number of features used: 30
Test Size: 0.35
Mean Sensitivity: 0.3439
Std Sensitivity: 0.0530
Mean Specificity: 0.9178
Std Specificity: 0.0123
Mean Balanced Accuracy: 0.6308
Std Balanced Accuracy: 0.0247
Number of features used: 30
Test Size: 0.30
Mean Sensitivity: 0.3424
Std Sensitivity: 0.0667
Mean Specificity: 0.9133
Std Specificity: 0.0119
Mean Balanced Accuracy: 0.6278
Std Balanced Ac

ValueError: The test_size = 1 should be greater or equal to the number of classes = 2

### Classification

In [12]:
from importlib import reload
reload(src)

<module 'src.functions' from 'c:\\Users\\User\\Documents\\GitHub\\Tese\\src\\functions.py'>

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
import scipy.signal as signal
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score, balanced_accuracy_score, confusion_matrix
import pandas as pd
from mrmr import mrmr_classif

selected_features = mrmr_classif(X=X_all, y=Y, K=30)
combined_df=pd.DataFrame()
for feature in selected_features:
    combined_df=pd.concat([combined_df, X_all[feature]], axis=1)
X=combined_df


param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf', 'poly']
}
grid = GridSearchCV(SVC(class_weight='balanced'), param_grid, refit=True, verbose=2, n_jobs=-1)
grid.fit(X, Y)

# Print the best parameters found
print(f"Best parameters found: {grid.best_params_}")

# Now use the best parameters for Stratified Shuffle Split
best_params = grid.best_params_
svm = SVC(class_weight='balanced', **best_params)

f1score = []
precision = []
recall = []
specificity = []
npv = []
bal_acc = []

splits = 100
sss = StratifiedShuffleSplit(n_splits=splits, test_size=0.3, random_state=0)
scaler = StandardScaler()
X_col = X.columns
Y = np.array(Y)

for train_index, test_index in sss.split(X, Y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    Y_train, Y_test = Y[train_index], Y[test_index]
    scal = scaler.fit(X_train)
    X_train = scal.transform(X_train)  # Variables standardization
    X_test = scal.transform(X_test)  # Variables standardization
    X_train = pd.DataFrame(X_train, columns=X_col)
    X_test = pd.DataFrame(X_test, columns=X_col)
    clf = svm.fit(X_train, Y_train)
    Y_predicted = clf.predict(X_test)
    f1score.append(f1_score(Y_test, Y_predicted))
    precision.append(precision_score(Y_test, Y_predicted))  # Precision = Positive predictive value
    npv.append(precision_score(Y_test, Y_predicted, pos_label=0))  # Negative predictive value
    recall.append(recall_score(Y_test, Y_predicted))  # Recall = Sensitivity
    specificity.append(recall_score(Y_test, Y_predicted, pos_label=0))
    bal_acc.append(balanced_accuracy_score(Y_test, Y_predicted))

print("Number of features used: "+ str(X.shape[1]))
print("Mean Sensitivity: {:.4f}".format(np.mean(recall)))
print("Std Sensitivity: {:.4f}".format(np.std(recall)))
print("Mean Specificity: {:.4f}".format(np.mean(specificity)))
print("Std Specificity: {:.4f}".format(np.std(specificity)))
print("Mean Balanced Accuracy: {:.4f}".format(np.mean(bal_acc)))
print("Std Balanced Accuracy: {:.4f}".format(np.std(bal_acc)))

In [5]:
from mrmr import mrmr_classif
i=10
while i <= 50:
    selected_features = mrmr_classif(X=X_all, y=Y, K=i)
    combined_df=pd.DataFrame()
    for feature in selected_features:
        combined_df=pd.concat([combined_df, X_all[feature]], axis=1)
    X=combined_df
    src.classification(i,X,Y)
    i=i+5
print("Done") #melhor antes foi 66.6%

100%|██████████| 10/10 [00:04<00:00,  2.48it/s]
